<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/finetune-gpt-oss-20b/notebooks/3-finetune_gpt_oss_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run CUREBench eval
- run prediction
- run eval

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
import os
import sys
import os.path
import glob

In [3]:
os.environ['BNB_CUDA_VERSION'] = '125'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5 min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [4]:
!git clone -b finetune-gpt-oss-20b https://github.com/christiejibaraki/CUREBench.git

fatal: destination path 'CUREBench' already exists and is not an empty directory.


In [5]:
%cd CUREBench

/content/CUREBench


In [6]:
!git pull

Already up to date.


In [7]:
!pip install virtualenv

In [8]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 228ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: Brotli==1.1.0, PyJWT==2.10.1, accelerate==1.10.1, acres==0.5.0, admet_ai==1.2.0, aiofiles==24.1.0, aiohappyeyeballs==2.6.1, aiohttp==3.13.0, aiosignal==1.4.0, alabaster==1.0.0, annotated_types==0.7.0, anyio==4.11.0, attrs==25.4.0, authlib==1.6.5, azure_ai_documentintelligence==1.0.2, azure_core==1.36.0, azure_identity==1.25.1, babel==2.17.0, beautifulsoup4==4.14.2, bitsandbytes==0.48.1, blinker==1.9.0, cachetools==6.2.1, certifi==2025.10.5, cffi==2.0.0, charset_normalizer==3.4.4, chemfunc==1.0.12, chemprop==1.6.1, ci_info==0.3.0, click==8.3.0, cloudpickle==3.1.1, cobble==0.1.4, coloredlogs==15.0.1, configobj==5.0.9, configparser==7.2.0, contourpy==1.3.3, cryptography==46.0.3, cycler==0.12.1, cyclopt

In [9]:
!./env/bin/pip list

Package                       Version
----------------------------- -----------
accelerate                    1.10.1
acres                         0.5.0
admet_ai                      1.2.0
aiofiles                      24.1.0
aiohappyeyeballs              2.6.1
aiohttp                       3.13.0
aiosignal                     1.4.0
alabaster                     1.0.0
annotated-types               0.7.0
anyio                         4.11.0
attrs                         25.4.0
Authlib                       1.6.5
azure-ai-documentintelligence 1.0.2
azure-core                    1.36.0
azure-identity                1.25.1
babel                         2.17.0
beautifulsoup4                4.14.2
bitsandbytes                  0.48.1
blinker                       1.9.0
Brotli                        1.1.0
cachetools                    6.2.1
certifi                       2025.10.5
cffi                          2.0.0
charset-normalizer            3.4.4
chemfunc                      1.0.12
chemp

In [10]:
!./env/bin/pip install -r requirements.txt

  Using cached pillow-12.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
  Using cached pydantic-2.12.2-py3-none-any.whl.metadata (85 kB)
  Using cached pydantic_core-2.41.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
Using cached pydantic-2.12.2-py3-none-any.whl (460 kB)
Using cached pillow-12.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (7.0 MB)
Using cached pydantic_core-2.41.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic_core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.10
    Uninstalling pydantic-2.11.10:
      

In [11]:
!./env/bin/pip install trl

In [12]:
!./env/bin/pip install trackio

  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached pydantic-2.11.10-py3-none-any.whl.metadata (68 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.6 MB)
Using cached pydantic-2.11.10-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.4
    Uninstalling pydantic_core-2.41.4:
      Successfully uninstalled pydantic_core-2.41.4
  Attempting uninstall: pillow
    Found existing installation: pillow 12.0.0
    Uninstalling pillow-12.0.0:
      Successfully uninstalled pillow-12.0.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.2
    Uninstalling pydantic-2.12.2:
      

In [13]:
# get sit packages path and add to sys.path
lib_path = glob.glob('./env/lib/python*')[0]
env_path = os.path.join(lib_path, 'site-packages')
if env_path not in sys.path:
    sys.path.append(env_path)
    print(f"Successfully added {env_path} to sys.path.")
else:
    print(f"{env_path} already in sys.path.")

Successfully added ./env/lib/python3.12/site-packages to sys.path.


### load dataset

In [14]:
from datasets import load_dataset

ds = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 19704
    })
})

In [16]:
train_data = ds['train']

In [17]:
train_data[0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

**system identity**

In [18]:
import json
from core.eval_framework import system_identity
print(system_identity)


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.


**convert single data example to harmony format**

In [19]:
def convert_to_harmony_format(data_item, system_prompt):
    """
    Convert a single data item to OpenAI Harmony format.

    Args:
        data_item: Dictionary with 'Question', 'Complex_CoT', and 'Response' keys
        system_prompt: System identity string

    Returns:
        Dictionary with 'messages' key containing the conversation
    """
    messages = [
        {'role': 'system',
            'content': system_prompt,
            'thinking': None},
        {'role': 'user',
            'content': data_item['Question'],
            'thinking': None},
        {'role': 'assistant',
            'content': data_item['Response'],
            'thinking': data_item['Complex_CoT']}
    ]
    return {'messages': messages}

In [20]:
converted_example = convert_to_harmony_format(train_data[0], system_identity)
print(json.dumps(converted_example, indent=2))

{
  "messages": [
    {
      "role": "system",
      "content": "\nYou are an expert medical assistant specializing in answering questions.\n\n**Your communication MUST strictly adhere to the Harmony channels:**\n1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.\n2.  **final:** Use this channel for the final output intended for the user.\n\n**Output Rule is Conditional:**\n* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.\n    * **Format:** `{\"answer\": \"<LETTER>\"}` (e.g., `{\"answer\": \"A\"}`)\n* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.\n\n**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.",
      "thinking": null
    },
    {
      "role

**convert training data**

In [21]:
formatted_training_data = [convert_to_harmony_format(item, system_identity) for item in train_data]

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

In [23]:
formatted_training_data[0]

{'messages': [{'role': 'system',
   'content': '\nYou are an expert medical assistant specializing in answering questions.\n\n**Your communication MUST strictly adhere to the Harmony channels:**\n1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.\n2.  **final:** Use this channel for the final output intended for the user.\n\n**Output Rule is Conditional:**\n* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.\n    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)\n* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.\n\n**Instruction:** Generate a complete response. The final output must use the \'final\' channel and adhere to the conditional format rule.',
   'thinking': None},
  {'role': 'user',
   'content': 'Given the sympt

In [24]:
conversation = tokenizer.apply_chat_template(formatted_training_data[0]['messages'], tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-10-31

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coh

### finetune setup

In [25]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

`torch_dtype` is deprecated! Use `dtype` instead!
This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### finetune with LoRA

In [26]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 15,040,512 || all params: 20,929,797,696 || trainable%: 0.0719


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


In [27]:
from trl import SFTConfig

training_args = SFTConfig(
    learning_rate=2e-4,
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_length=2048,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    output_dir="gpt-oss-20b-medical-reasoner",
    report_to="trackio",
    push_to_hub=True,
)

In [28]:
from trl import SFTTrainer
from datasets import Dataset

# Convert your list to HF Dataset
training_data_formatted = Dataset.from_list(formatted_training_data)

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=training_data_formatted,
    processing_class=tokenizer,
)
trainer.train()

Tokenizing train dataset:   0%|          | 0/19704 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/19704 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998}.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: cibaraki/trackio-dataset
* Creating new space: https://huggingface.co/spaces/cibaraki/trackio
* View dashboard by going to: https://cibaraki-trackio.hf.space/


* Created new run: cibaraki-1761953180


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.85 GiB. GPU 0 has a total capacity of 39.56 GiB of which 2.55 GiB is free. Process 179908 has 37.00 GiB memory in use. Of the allocated memory 36.40 GiB is allocated by PyTorch, and 102.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### load curebench val dataset

In [ ]:
import pandas as pd
from typing import Dict, List
from torch.utils.data import DataLoader
from dataset_utils import build_dataset
from core.eval_framework import GPTOSS20BModel
from openai_harmony import load_harmony_encoding

In [ ]:
def load_dataset_by_config(config_path):
  # load config file to get dataset info
  config = json.load(open(config_path, 'r')) if config_path else {}
  if 'dataset' in config:
      dataset_config = config['dataset']
      dataset_name = dataset_config.get('dataset_name', 'treatment')
  print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
  dataset_path = dataset_config.get("dataset_path")

  # build dataset
  dataset = build_dataset(
        dataset_config.get("dataset_path"),
    )
  dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
  dataset_list = []

  for batch in dataloader:
      question_type = batch[0][0]

      if question_type == "multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
      elif question_type == "open_ended_multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
              "meta_question": batch[4][0],
          })
      elif question_type == "open_ended":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
  return dataset_list

In [ ]:
val_data_config_path= "metadata_config_val.json"
val_data_list = load_dataset_by_config(val_data_config_path)

In [ ]:
dataset = val_data_list[:10]

In [ ]:
dataset

In [ ]:
example = dataset[3]
print(example)

In [ ]:
question = example["question"]
question_type = example["question_type"]

In [ ]:
# Format prompt
if question_type == "multi_choice":
    prompt = f"The following is a multiple choice question about medicine. Answer with a valid json containing the letter corresponding to the correct answer.\n\nQuestion: {question}\n\nAnswer:"
elif question_type == "open_ended_multi_choice" or question_type == "open_ended":
    prompt = f"The following is an open-ended question about medicine. Provide a comprehensive answer.\n\nQuestion: {question}\n\nAnswer:"

In [ ]:
print(f"prompt: {prompt}")